In [2]:
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
# Generate a dictionary of Unit-norm vectors

def generate_gaussian_noises_dict(N, d):
    gaussian_noises = np.random.normal(size=(d, N))
    norms = np.linalg.norm(gaussian_noises, axis=0, keepdims=True)
    # Create unit-norm vectors
    unit_vectors = gaussian_noises / norms
    return unit_vectors

def generate_sparse_response(gaussian_matrix, m):
    indices = np.random.choice(gaussian_matrix.shape[1], size=m, replace=False)
    selected_vectors = gaussian_matrix[:, indices]
    coefficients = np.random.normal(size=(m, 1))  # random coefficients for each selected vector
    y = selected_vectors @ coefficients
    return y, indices, coefficients

In [4]:
# Use the functions to generate a Gaussian noise matrix and a sparse response
np.random.seed(0)
N = 100000
d = 300
m = 2
gaussian_noises_matrix = generate_gaussian_noises_dict(N, d)
y, indices, coefficients = generate_sparse_response(gaussian_noises_matrix, m)

In [5]:
coefficients

array([[0.34249941],
       [1.7701584 ]])

In [6]:
indices

array([16274, 83612])

In [19]:
# Matching Pursuit

def matching_pursuit(s, phi, K):
    """
    Perform the Matching Pursuit algorithm

    Args:
    s (numpy.ndarray): Input signal
    phi (numpy.ndarray): Dictionary
    K (int): Number of iterations (sparsity)

    Returns:
    a (numpy.ndarray): Sparse representation of s
    indices (list): List of indices of selected atoms
    coefficients (list): List of coefficients of selected atoms
    """
    # Initialize a and r
    a = np.zeros_like(s)
    r = s.copy()
    indices = []
    coefficients = []
    # Perform Matching Pursuit
    for _ in range(K):
        # Compute inner products
        inner_products = phi.T @ r

        # Find the index with maximum absolute correlation
        lambda_k = np.argmax(np.abs(inner_products), axis=0)

        # Save the index
        indices.append(lambda_k[0])

        # Save the coefficient
        coefficients.append(inner_products[lambda_k])

        # Update a
        a += coefficients[-1] * phi[:, lambda_k]

        # Update r
        r = s - a

    return a, indices, coefficients

# Perform Matching Pursuit
MP_residual, MP_indices, MP_coefficients = matching_pursuit(y, gaussian_noises_matrix, 2*m)

In [20]:
MP_coefficients

[array([[1.77858963]]),
 array([[0.34229186]]),
 array([[-0.00842612]]),
 array([[0.00020742]])]

In [21]:
MP_indices

[83612, 16274, 83612, 16274]

In [34]:
# Orthogonal Matching Pursuit

def orthogonal_matching_pursuit(s, phi, K):
    """
    Perform the Orthogonal Matching Pursuit algorithm

    Args:
    s (numpy.ndarray): Input signal
    phi (numpy.ndarray): Dictionary
    K (int): Number of iterations (sparsity)

    Returns:
    a (numpy.ndarray): Sparse representation of s
    indices (list): Indices of the selected atoms
    coefficients (list): Coefficients of the selected atoms
    """
    # Initialize a and r
    a = np.zeros_like(s)
    r = s.copy()
    indices = []
    coefficients = []

    for _ in range(K):
        # Compute inner products
        inner_products = phi.T @ r
        
        # Though paper says OMP will not choose the same index twice, it does.
        inner_products[indices] = np.min(np.abs(inner_products))

        # Find the index with maximum absolute correlation
        lambda_k = np.argmax(np.abs(inner_products), axis=0)
        # print(np.max(np.abs(inner_products)))
        
        # Save the index
        indices.append(lambda_k[0])
        # print(indices)

        # Ordinary Least Squares
        X = phi[:, indices]
        betas = np.linalg.inv(X.T @ X) @ X.T @ s

        # Save the coefficient
        coefficients = betas

        # Update a
        a = X @ betas

        # Update r
        r = s - a

    return a, indices, coefficients

# Perform Orthogonal Matching Pursuit
OMP_residual, OMP_indices, OMP_coefficients = orthogonal_matching_pursuit(y, gaussian_noises_matrix, 2*m)

In [29]:
OMP_coefficients

array([[ 1.77015840e+00],
       [ 3.42499410e-01],
       [ 1.35308431e-16],
       [-6.93889390e-17]])

In [26]:
OMP_indices

[83612, 16274, 20541, 65973]

In [168]:
# Weak Orthogonal Matching Pursuit

def weak_orthogonal_matching_pursuit(s, phi, alpha):
    """
    Perform the Weak Orthogonal Matching Pursuit algorithm

    Args:
    s (numpy.ndarray): Input signal
    phi (numpy.ndarray): Dictionary
    alpha (float): Threshold for stopping the algorithm

    Returns:
    a (numpy.ndarray): Sparse representation of s
    indices (list): Indices of the selected atoms
    coefficients (list): Coefficients of the selected atoms
    """

    # Initialize a and r
    a = np.zeros_like(s)
    r = s.copy()
    indices = []
    coefficients = []
    flag = True

    while flag:
        # Compute inner products
        inner_products = phi.T @ r

        # Create a copy of inner_products to find out the largest inner product
        # Though paper says OMP will not choose the same index twice, it does.
        inner_products_copy = inner_products.copy()
        inner_products_copy[indices] = np.min(np.abs(inner_products_copy))

        # Find the index with maximum absolute correlation
        lambda_k = np.argmax(np.abs(inner_products_copy), axis=0)

        # Save the index
        indices.append(lambda_k[0])

        # Ordinary Least Squares
        X = phi[:, indices]
        betas = np.linalg.inv(X.T @ X) @ X.T @ s

        # Save the coefficient
        coefficients = betas

        # Update a
        a = X @ betas

        # Update r
        r = s - a

        # Check if the largest inner product is greater than alpha times the largest inner product
        largest_inner_product = np.abs(inner_products[lambda_k][0][0])
        if largest_inner_product >= alpha * np.max(np.abs(inner_products)):
            flag = False
    return a, indices, coefficients

# Perform Weak Orthogonal Matching Pursuit
WOMP_residual, WOMP_indices, WOMP_coefficients = weak_orthogonal_matching_pursuit(y_train, X_train, 0.8)

In [14]:
# LASSO Regression
np.random.seed(0)

# Reduced Standardization because the data is already unit-norm

lasso = Lasso()
LASSO_alphas = {"alpha": [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]}
LASSO_cv = KFold(n_splits=10, shuffle=True, random_state=0)

# Perform grid search on training data
grid = GridSearchCV(lasso, LASSO_alphas, scoring='neg_mean_squared_error', cv=LASSO_cv, return_train_score=True)
grid.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print('Best parameters:', grid.best_params_)
print('Best score (neg_mean_squared_error):', grid.best_score_)

# Evaluate the model on the testing set
y_pred = grid.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
print('Testing set MSE:', mse_test)

Best parameters: {'alpha': 0.0001}
Best score (neg_mean_squared_error): -6.0769806502645135e-06
Testing set MSE: 5.920689195738421e-06


In [97]:
grid.best_estimator_.coef_

array([-0., -0., -0., ...,  0.,  0., -0.])